In [ ]:
import numpy as np
import torch
import tdg
from tdg import EffectiveRangeExpansion
from tdg import ReducedTwoBodyA1Hamiltonian as A1
from tdg.Luescher import Zeta2D

import matplotlib.pyplot as plt

In [ ]:
torch.set_printoptions(precision=8)
Z = Zeta2D()

To go towards the continuum limit we want to hold the scattering fixed

In [ ]:
ere = EffectiveRangeExpansion(torch.tensor([1.0]))

and describe what interaction we want to use

In [ ]:
radii = [[0,0]]

Let's tune many discretizations!

In [ ]:
# Fewer will finish faster, obviously!
# NX = np.arange(7, 13, 2)   # Takes about 5 minutes on my laptop
NX = np.arange(7, 29, 2)   # Takes about 18 minutes on my laptop

and store computed results in a dictionary.

In [ ]:
tuning = {
    #  For a starting guess for nx we'll use the results for nx-2.
    #  I have a starting guess for 7, courtesy of Mathematica.
    5: tdg.Tuning(ere, tdg.Lattice(5), radii, C=torch.tensor([-5.]))
}

In [ ]:
%%time
for nx in NX:
    if nx in tuning:
        continue
    tuning[nx] = tdg.Tuning(ere, tdg.Lattice(nx), radii, starting_guess=tuning[nx-2].C)
    print(f"{nx=}: {tuning[nx]}:")
    print(f"        {tuning[nx].C}")

We process these coefficients into energy eigenvalues and dimensionless x.

x goes through the Luescher zeta function.

In [ ]:
H = {nx: A1(tdg.Lattice(nx), [tdg.LegoSphere(r) for r in radii]) for nx in NX}
E = {nx: H[nx].eigenenergies(tuning[nx].C) for nx in NX}
x = {nx: E[nx] * nx**2 / (2*torch.pi)**2 for nx in NX}
z = {nx: Z(x[nx]) / torch.pi**2 for nx in NX}

Now we can visualize how well our tuning did!

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,8))


# First draw the zeta function.
exact = torch.linspace(-5.001, 25.001, 1000)
Z.plot(ax, exact, color='gray')

# and the analytic piece of the ERE
ax.plot(exact, ere.analytic(exact).clone().detach(), color='black', linestyle='dashed')

# then, for each spatial discretization
for nx in NX:
    ax.plot(
        x[nx].clone().detach().numpy(),
        z[nx].clone().detach().numpy(),
        linestyle='none', marker='o',
        label=f'{nx=}'
    )

ax.set_xlim([-5,25])
ax.set_ylim([-1,1])
ax.legend();

Looks like the two-body energy levels are converging as expected!

In [ ]:
levels = [1, 2, 3, 4, 5, 6]

fig, axs = plt.subplots(len(levels), 1, figsize=(12, 8 * len(levels)), squeeze=False, sharex=True)
axs = [ax[0] for ax in axs]

for level, ax in zip(levels, axs):
    for nx in NX:
        ax.plot(
            [1/nx**2],
            [x[nx][level].clone().detach().numpy()],
            marker='o'
        )
    ax.set_xlabel(r'$1/N_x^2$')
    ax.set_ylabel(r'$x$')

axs[0].set_xlim([0, 1.1/min(NX)**2]);

The convergence appears to be QUADRATIC with nx!

How does the coefficient approach the continuum?

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,8))

CatalanG = 0.9159655941772190

nxInverse=torch.linspace(0, 0.2, 1000)
amplitudeAnswer = (-2 * torch.pi / (torch.log(0.5 * ere.a / nxInverse) + torch.log(torch.tensor(2)) - 2 * CatalanG / torch.pi))

ax.plot(nxInverse.detach(), amplitudeAnswer.detach(), color='black')

for nx in NX:
    ax.plot([1/nx], [tuning[nx].C.detach().numpy()],
        marker='o'
        )

ax.set_xlabel(r"$1/N_x$")
ax.set_ylabel(r"$C_0$");

Let's try to visualize the convergence of the states.  

We'll take as reference the coarsest lattice we tuned and look at ratios of eigenenergies compared to that lattice's energies.

In [ ]:
levels = [0, 1, 2, 3, 4, 5, 6]

fig, ax = plt.subplots(2, 1, figsize=(12, 8), squeeze=True, sharex=True)

reference = min(NX)
for level in levels:
    ax[0].plot(
        (1/NX**2),
        [(E[nx][level]/E[reference][level]).detach().numpy() for nx in NX],
        marker='o',
        linestyle='none'
    )
    ax[1].plot(
        (1/NX**2),
        [(x[nx][level]/x[reference][level]).detach().numpy() for nx in NX],
        marker='o',
        linestyle='none',
        label=f'{level=}'
    )
    
ax[0].set_xlim([0, 1.1/min(NX)**2]);
ax[0].plot([0, 1/min(NX)**2], [0,1], label=r'perfect $N_{x}^{-2}$');
ax[0].set_ylim([0, 1.1]);
ax[0].set_ylabel(r'$E_{N_x}/E_'+f'{reference}'+r'$')
ax[1].set_ylabel(r'$x_{N_x}/x_'+f'{reference}'+r'$')
ax[1].set_xlabel(r'$1/N_x^2$')
ax[0].legend()
ax[1].legend()

We see that all energies go to 0 like $N_x^{-2}$ as we head towards the $N_x=\infty$ limit, while the Lüscher $x$ go to their nonvanishing continuum value the same way.  (The $0^{th}$ level stays fixed trivially with no corrections, as that's what we tuned!)